In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive


In [5]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 299, 299
train_data_dir = "drive/MO444/train"
validation_data_dir = "drive/MO444/val"
batch_size = 10
epochs = 5

model = applications.InceptionResNetV2(weights='imagenet', include_top=False,input_shape = (img_width, img_height, 3))

 62947328/219055592 [=======>......................] - ETA: 1:06

219062272/219055592 [==============================] - 58s 0us/step


In [0]:
# Freeze the layers which we don't want to train.
for layer in model.layers:
    layer.trainable = False

In [7]:
#Adding custom Layers 

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(83, activation="softmax")(x)

# creating the final model 

model_final = Model(input = model.input, output = predictions)

# compile the model 

model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["acc"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [8]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 8300 images belonging to 83 classes.
Found 6024 images belonging to 83 classes.


In [9]:
# Save the model according to the conditions  
#checkpoint = ModelCheckpoint("resnet_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


# Train the model 
model_final.fit_generator(
train_generator,
steps_per_epoch=train_generator.samples/train_generator.batch_size,
epochs = epochs,
validation_data = validation_generator,
validation_steps=validation_generator.samples/validation_generator.batch_size,
verbose=1)

model_final.save('drive/MO444/model.h5')

Epoch 1/5
830/830 [==============================] - 8965s 11s/step - loss: 2.2733 - acc: 0.4882 - val_loss: 0.2595 - val_acc: 0.9290
Epoch 2/5
102/830 [==>...........................] - ETA: 4:49 - loss: 1.3073 - acc: 0.6686

830/830 [==============================] - 539s 650ms/step - loss: 1.2120 - acc: 0.6894 - val_loss: 0.2353 - val_acc: 0.9344
Epoch 3/5
189/830 [=====>........................] - ETA: 4:37 - loss: 1.1036 - acc: 0.7101

830/830 [==============================] - 545s 657ms/step - loss: 1.0762 - acc: 0.7210 - val_loss: 0.2355 - val_acc: 0.9339
Epoch 4/5
222/830 [=======>......................] - ETA: 4:18 - loss: 1.0126 - acc: 0.7248

830/830 [==============================] - 550s 662ms/step - loss: 1.0047 - acc: 0.7357 - val_loss: 0.2449 - val_acc: 0.9326
Epoch 5/5
234/830 [=======>......................] - ETA: 4:14 - loss: 0.9885 - acc: 0.7389

830/830 [==============================] - 550s 663ms/step - loss: 0.9461 - acc: 0.7475 - val_loss: 0.2407 - val_acc: 0.9359


In [3]:
from keras.models import load_model

model = load_model('drive/MO444/model.h5')

Using TensorFlow backend.


In [8]:
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 299, 299
train_data_dir = "drive/MO444/train"
validation_data_dir = "drive/MO444/val"
test_data_dir = "drive/MO444/test_classe"
batch_size = 10
epochs = 5


datagen = ImageDataGenerator(
rescale = 1./255)

train_generator = datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical",
shuffle=False)

validation_generator = datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical",
batch_size = batch_size, 
shuffle=False)

test_generator = datagen.flow_from_directory(
test_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical",
batch_size = batch_size, 
shuffle=False)

Found 8300 images belonging to 83 classes.
Found 6024 images belonging to 83 classes.
Found 5145 images belonging to 83 classes.


In [9]:
prediction_train = model.predict_generator(train_generator,steps=train_generator.samples/train_generator.batch_size,verbose=1)
prediction_val = model.predict_generator(validation_generator,steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
prediction_test = model.predict_generator(test_generator,steps=test_generator.samples/test_generator.batch_size,verbose=1)

505/602 [========================>.....] - ETA: 9:40

515/514 [==============================] - 3844s 7s/step


In [0]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score


def calc_score(prediction, generator, name):
  pred_class = np.argmax(prediction, axis=1)
  label_class = generator.classes
  f1 = f1_score(pred_class, label_class, average='weighted')
  acc_norm = accuracy_score(pred_class, label_class) 
  print('# %s'%name)
  print('F1 score: %f'%f1)
  print('Normalized score: %f\n'%acc_norm)

In [11]:
calc_score(prediction_train,train_generator,'Treino')
calc_score(prediction_val,validation_generator,'Validacao')
calc_score(prediction_test, test_generator, 'Teste')


# Treino
F1 score: 0.940342
Normalized score: 0.940241

# Validacao
F1 score: 0.935985
Normalized score: 0.935923

# Teste
F1 score: 0.029145
Normalized score: 0.029543



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [75]:
from keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 299, 299
train_data_dir = "drive/MO444/train"
validation_data_dir = "drive/MO444/val"
test_data_dir = "drive/MO444/test_classe"
batch_size = 10
epochs = 5

datagen = ImageDataGenerator(
rescale = 1./255)


test_generator = datagen.flow_from_directory(
test_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical",
batch_size = batch_size, 
shuffle=False)

Found 5328 images belonging to 83 classes.


In [76]:
prediction_test = model.predict_generator(test_generator,steps=test_generator.samples/test_generator.batch_size,verbose=1)

533/532 [==============================] - 4875s 9s/step


In [77]:
calc_score(prediction_test, test_generator, 'Teste')

# Teste
F1 score: 0.946902
Normalized score: 0.944257



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [74]:
np.argmax(prediction_test[5071])

4

In [57]:
for i in range(566,636):
  print(np.argmax(prediction_test[i]))

16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
52
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16
16


In [56]:
len(list(filter(lambda x: x==8, test_generator.classes)))

70